In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.common.by import By

import re
import os
import requests
import urllib.request
import datetime
import glob
import time
import csv
import cmd
import sys
from random import seed
from random import random
import numpy as np
import pickle
import pandas as pd
import json

options = Options()
options.headless = False
options.add_argument('log-level=2')
options.add_argument(
    '--ignore-certificate-errors')  # https://stackoverflow.com/questions/37883759/errorssl-client-socket-openssl-cc1158-handshake-failed-with-chromedriver-chr
options.add_argument(
    '--ignore-ssl-errors')  # https://stackoverflow.com/questions/37883759/errorssl-client-socket-openssl-cc1158-handshake-failed-with-chromedriver-chr
options.add_argument("--start-maximized")
options.add_argument("--incognito")
# Bron: https://stackoverflow.com/questions/53998690/downloading-a-pdf-using-selenium-chrome-and-python
options.add_experimental_option('prefs', {
    "download.default_directory": "/Users/mennohuijbregts/Downloads",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
})

In [434]:
driver = webdriver.Chrome(options=options)

In [442]:
date_excepted = [20150917]


dates_to_retrieve = json.load(open('dates.json'))
dates_retrieved = json.load(open('data/dates_retrieved.json'))

with open('data/price.pkl', 'rb') as f:
    price_dict = pickle.load(f)
with open('data/supply.pkl', 'rb') as f:
    supply_dict = pickle.load(f)
with open('data/volume.pkl', 'rb') as f:
    volume_dict = pickle.load(f)    
with open('data/marketcap.pkl', 'rb') as f:
    marketcap_dict = pickle.load(f)
with open('data/index.pkl', 'rb') as f:
    index_dict = pickle.load(f)

for (count, date) in enumerate(dates_to_retrieve):
    y = 1000
    if date not in dates_retrieved and date not in date_excepted:
        try:
            print(f'start - {date}')
            url = 'https://coinmarketcap.com/historical/'+str(date)
            driver.get(url)

            for r in range(0, 200):
                mytable = driver.find_elements_by_class_name("cmc-table-row")
                columns = mytable[r].find_elements_by_css_selector('td')

                while len(columns) < 4:
                    seed(1)
                    y = y+500+random()
                    driver.execute_script("window.scrollTo(0, "+str(y)+")")

                    time.sleep(.03)

                    mytable = driver.find_elements_by_class_name("cmc-table-row")
                    columns = mytable[r].find_elements_by_css_selector('td')

                index = columns[0].text
                ticker = columns[2].text + '-' + columns[1].text
                price = columns[4].text[1:]
                marketcap = columns[3].text[1:]
                volume = columns[6].text[1:]
                supply = columns[5].text

                if ticker in price_dict.keys():
                    price_dict[ticker] = np.append(price_dict[ticker][:len(dates_retrieved)], price)
                    volume_dict[ticker] = np.append(volume_dict[ticker][:len(dates_retrieved)], volume)
                    marketcap_dict[ticker] = np.append(marketcap_dict[ticker][:len(dates_retrieved)], marketcap)
                    supply_dict[ticker] = np.append(supply_dict[ticker][:len(dates_retrieved)], supply)
                    index_dict[ticker] = np.append(index_dict[ticker][:len(dates_retrieved)], index)
                else:
                    nan_array = np.empty([1,len(dates_retrieved)])
                    if len(nan_array) > 0: nan_array[:] = np.nan
                    price_dict[ticker] = nan_array
                    volume_dict[ticker] = nan_array
                    marketcap_dict[ticker] = nan_array
                    supply_dict[ticker] = nan_array
                    index_dict[ticker] = nan_array
                    
                    price_dict[ticker] = np.append(price_dict[ticker][:len(dates_retrieved)], price)
                    volume_dict[ticker] = np.append(volume_dict[ticker][:len(dates_retrieved)], volume)
                    marketcap_dict[ticker] = np.append(marketcap_dict[ticker][:len(dates_retrieved)], marketcap)
                    supply_dict[ticker] = np.append(supply_dict[ticker][:len(dates_retrieved)], supply)
                    index_dict[ticker] = np.append(index_dict[ticker][:len(dates_retrieved)], index)


            # Add date to dates retrieved.      
            dates_retrieved.append(date)
            print(f'end - {date}')
        except:
            print(f"Error occured at {date}")
            break


        # Add NaN for missing days
        for i in price_dict:
            if len(price_dict[i]) < len(dates_retrieved):
                price_dict[i] = np.append(price_dict[i], np.nan)
                volume_dict[i] = np.append(volume_dict[i], np.nan)
                supply_dict[i] = np.append(supply_dict[i], np.nan)
                marketcap_dict[i] = np.append(marketcap_dict[i], np.nan)
                index_dict[i] = np.append(index_dict[i], np.nan)
            elif len(price_dict[i]) > len(dates_retrieved):
                print('-------SOMETHING WRONG -------')

        # Save as pickle
        if count%5 == 0:
            with open('data/price.pkl', 'wb') as handle:
                pickle.dump(price_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('data/supply.pkl', 'wb') as handle:
                pickle.dump(supply_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('data/volume.pkl', 'wb') as handle:
                pickle.dump(volume_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('data/marketcap.pkl', 'wb') as handle:
                pickle.dump(marketcap_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('data/index.pkl', 'wb') as handle:
                pickle.dump(index_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('data/dates_retrieved.json', 'w') as fp:
                json.dump(dates_retrieved, fp,  indent=4)
            print('saved')
print('Done')

start - 20180525
end - 20180525
start - 20180526
end - 20180526
start - 20180527
end - 20180527
start - 20180528
end - 20180528
start - 20180529
end - 20180529
saved
start - 20180530
end - 20180530
start - 20180531
end - 20180531
start - 20180601
end - 20180601
start - 20180602
end - 20180602
start - 20180603
end - 20180603
saved
start - 20180604
end - 20180604
start - 20180605
end - 20180605
start - 20180606
end - 20180606
start - 20180607
end - 20180607
start - 20180608
end - 20180608
saved
start - 20180609
end - 20180609
start - 20180610
end - 20180610
start - 20180611
end - 20180611
start - 20180612
end - 20180612
start - 20180613
end - 20180613
saved
start - 20180614
end - 20180614
start - 20180615
end - 20180615
start - 20180616
end - 20180616
start - 20180617
end - 20180617
start - 20180618
end - 20180618
saved
start - 20180619
end - 20180619
start - 20180620
end - 20180620
start - 20180621
end - 20180621
start - 20180622
end - 20180622
start - 20180623
end - 20180623
saved
star

start - 20190127
end - 20190127
start - 20190128
end - 20190128
start - 20190129
end - 20190129
saved
start - 20190130
end - 20190130
start - 20190131
end - 20190131
start - 20190201
end - 20190201
start - 20190202
end - 20190202
start - 20190203
end - 20190203
saved
start - 20190204
end - 20190204
start - 20190205
end - 20190205
start - 20190206
end - 20190206
start - 20190207
end - 20190207
start - 20190208
end - 20190208
saved
start - 20190209
end - 20190209
start - 20190210
end - 20190210
start - 20190211
end - 20190211
start - 20190212
end - 20190212
start - 20190213
end - 20190213
saved
start - 20190214
end - 20190214
start - 20190215
end - 20190215
start - 20190216
end - 20190216
start - 20190217
end - 20190217
start - 20190218
end - 20190218
saved
start - 20190219
end - 20190219
start - 20190220
end - 20190220
start - 20190221
end - 20190221
start - 20190222
end - 20190222
start - 20190223
end - 20190223
saved
start - 20190224
end - 20190224
start - 20190225
end - 20190225
star

start - 20191001
end - 20191001
saved
start - 20191002
end - 20191002
start - 20191003
end - 20191003
start - 20191004
end - 20191004
start - 20191005
end - 20191005
start - 20191006
end - 20191006
saved
start - 20191007
end - 20191007
start - 20191008
end - 20191008
start - 20191009
end - 20191009
start - 20191010
end - 20191010
start - 20191011
end - 20191011
saved
start - 20191012
end - 20191012
start - 20191013
end - 20191013
start - 20191014
end - 20191014
start - 20191015
end - 20191015
start - 20191016
end - 20191016
saved
start - 20191017
end - 20191017
start - 20191018
end - 20191018
start - 20191019
end - 20191019
start - 20191020
end - 20191020
start - 20191021
end - 20191021
saved
start - 20191022
end - 20191022
start - 20191023
end - 20191023
start - 20191024
end - 20191024
start - 20191025
end - 20191025
start - 20191026
end - 20191026
saved
start - 20191027
end - 20191027
start - 20191028
end - 20191028
start - 20191029
end - 20191029
start - 20191030
end - 20191030
star

start - 20200604
end - 20200604
start - 20200605
end - 20200605
start - 20200606
end - 20200606
saved
start - 20200607
end - 20200607
start - 20200608
end - 20200608
start - 20200609
end - 20200609
start - 20200610
end - 20200610
start - 20200611
end - 20200611
saved
start - 20200612
end - 20200612
start - 20200613
end - 20200613
start - 20200614
end - 20200614
start - 20200615
end - 20200615
start - 20200616
end - 20200616
saved
start - 20200617
end - 20200617
start - 20200618
end - 20200618
start - 20200619
end - 20200619
start - 20200620
end - 20200620
start - 20200621
end - 20200621
saved
start - 20200622
end - 20200622
start - 20200623
end - 20200623
start - 20200624
end - 20200624
start - 20200625
end - 20200625
start - 20200626
end - 20200626
saved
start - 20200627
end - 20200627
start - 20200628
end - 20200628
start - 20200629
end - 20200629
start - 20200630
end - 20200630
start - 20200701
end - 20200701
saved
start - 20200702
end - 20200702
start - 20200703
end - 20200703
star

In [431]:
dates_retrieved

[20140731,
 20140801,
 20140802,
 20140803,
 20140804,
 20140805,
 20140806,
 20140807,
 20140808,
 20140809,
 20140810,
 20140811,
 20140812,
 20140813,
 20140814,
 20140815,
 20140816,
 20140817,
 20140818,
 20140819,
 20140820,
 20140821,
 20140822,
 20140823,
 20140824,
 20140825,
 20140826,
 20140827,
 20140828,
 20140829,
 20140830,
 20140831,
 20140901,
 20140902,
 20140903,
 20140904,
 20140905,
 20140906,
 20140907,
 20140908,
 20140909,
 20140910,
 20140911,
 20140912,
 20140913,
 20140914,
 20140915,
 20140916,
 20140917,
 20140918,
 20140919,
 20140920,
 20140921,
 20140922,
 20140923,
 20140924,
 20140925,
 20140926,
 20140927,
 20140928,
 20140929,
 20140930,
 20141001,
 20141002,
 20141003,
 20141004,
 20141005,
 20141006,
 20141007,
 20141008,
 20141009,
 20141010,
 20141011,
 20141012,
 20141013,
 20141014,
 20141015,
 20141016,
 20141017,
 20141018,
 20141019,
 20141020,
 20141021,
 20141022,
 20141023,
 20141024,
 20141025,
 20141026,
 20141027,
 20141028,
 20141029,

In [6]:
cat = ['price', 'volume', 'supply', 'marketcap', 'index']
for c in cat:
    with open(f'data/{c}.pkl', 'rb') as f:
            dict_ = pickle.load(f)
    print(dict_)
    dates_retrieved = json.load(open('data/dates_retrieved.json'))


    columns_ = []
    data_ = []
    data_.append(dates_retrieved)
    columns_.append('date')
    for i in dict_:
        data_.append(dict_[i])
        columns_.append(i)

    df = pd.DataFrame(data_,index=columns_).T.set_index('date')
    df.to_csv(F'data/{c}.csv')

{'BTC-Bitcoin': array(['586.23', '594.92', '589.33', ..., '11,754.05', '11,675.74',
       '11,878.11'], dtype='<U32'), 'LTC-Litecoin': array(['7.54', '7.70', '7.63', ..., '58.13', '57.14', '58.29'],
      dtype='<U32'), 'NXT-Nxt': array(['0.04513', '0.04458', '0.04595', ..., 'nan', 'nan', 'nan'],
      dtype='<U32'), 'XRP-XRP': array(['0.005189', '0.005369', '0.005294', ..., '0.294', '0.2887',
       '0.2947'], dtype='<U32'), 'DASH-Dash': array(['5.57', '5.64', '5.71', ..., '98.07', '97.64', '95.66'],
      dtype='<U32'), 'PPC-Peercoin': array(['1.16', '1.19', '1.15', ..., 'nan', 'nan', 'nan'], dtype='<U32'), 'BTS-BitShares': array(['0.0099', '0.009587', '0.009956', ..., '0.0262', '0.02617',
       '0.02751'], dtype='<U32'), 'DOGE-Dogecoin': array(['0.0002093', '0.0002109', '0.0002074', ..., '0.00351', '0.003463',
       '0.003499'], dtype='<U32'), 'NMC-Namecoin': array(['1.56', '1.59', '1.56', ..., 'nan', 'nan', 'nan'], dtype='<U32'), 'MAID-MaidSafeCoin': array(['0.0277', '0.02566', 

{'BTC-Bitcoin': array(['22,467,894.00', '18,215,930.00', '8,359,081.00', ...,
       '17,572,057,837.41', '17,489,608,833.29', '26,114,112,569.42'],
      dtype='<U32'), 'LTC-Litecoin': array(['3,047,706.75', '1,849,424.13', '1,113,909.63', ...,
       '1,726,820,322.75', '1,601,010,109.77', '2,247,183,310.28'],
      dtype='<U32'), 'NXT-Nxt': array(['46,166.01', '36,561.59', '71,648.32', ..., 'nan', 'nan', 'nan'],
      dtype='<U32'), 'XRP-XRP': array(['572,621.06', '393,153.47', '220,517.88', ..., '1,355,768,291.29',
       '1,391,288,258.62', '2,108,627,559.93'], dtype='<U32'), 'DASH-Dash': array(['387,401.03', '252,697.97', '185,190.22', ..., '355,970,818.28',
       '314,807,529.78', '306,005,368.51'], dtype='<U32'), 'PPC-Peercoin': array(['173,546.72', '83,444.05', '77,512.30', ..., 'nan', 'nan', 'nan'],
      dtype='<U32'), 'BTS-BitShares': array(['285,239.16', '136,484.17', '150,360.61', ..., '5,560,260.92',
       '5,295,416.55', '12,170,331.47'], dtype='<U32'), 'DOGE-Dogecoin

{'BTC-Bitcoin': array(['13,084,925 BTC', '13,088,900 BTC', '13,093,075 BTC', ...,
       '18,455,275 BTC', '18,456,250 BTC', '18,457,093 BTC'], dtype='<U32'), 'LTC-Litecoin': array(['30,722,504 LTC', '30,751,354 LTC', '30,778,654 LTC', ...,
       '65,197,748 LTC', '65,205,660 LTC', '65,213,585 LTC'], dtype='<U32'), 'NXT-Nxt': array(['999,997,096 NXT *', '999,997,096 NXT *', '999,997,096 NXT *', ...,
       'nan', 'nan', 'nan'], dtype='<U32'), 'XRP-XRP': array(['8,252,600,677 XRP *', '8,252,600,677 XRP *',
       '8,252,600,677 XRP *', ..., '44,862,646,997 XRP *',
       '44,862,646,997 XRP *', '44,862,646,997 XRP *'], dtype='<U32'), 'DASH-Dash': array(['4,519,974 DASH', '4,523,397 DASH', '4,526,471 DASH', ...,
       '9,643,756 DASH', '9,645,311 DASH', '9,646,900 DASH'], dtype='<U32'), 'PPC-Peercoin': array(['21,598,291 PPC', '21,601,351 PPC', '21,604,292 PPC', ..., 'nan',
       'nan', 'nan'], dtype='<U32'), 'BTS-BitShares': array(['1,999,775,140 BTS *', '1,999,775,140 BTS *',
      

{'BTC-Bitcoin': array(['7,670,835,225.22', '7,786,790,644.73', '7,716,182,869.54', ...,
       '216,924,147,850.74', '215,490,364,254.78', '219,235,405,395.42'],
      dtype='<U32'), 'LTC-Litecoin': array(['231,571,707.45', '236,842,447.33', '234,697,452.99', ...,
       '3,789,893,300.92', '3,725,788,181.86', '3,801,224,616.55'],
      dtype='<U32'), 'NXT-Nxt': array(['45,126,046.32', '44,584,122.47', '45,952,931.71', ..., 'nan',
       'nan', 'nan'], dtype='<U32'), 'XRP-XRP': array(['42,821,612.60', '44,307,695.37', '43,690,553.70', ...,
       '13,190,586,826.83', '12,949,750,916.51', '13,219,165,921.51'],
      dtype='<U32'), 'DASH-Dash': array(['25,186,493.59', '25,521,300.11', '25,837,964.97', ...,
       '945,779,284.79', '941,794,717.67', '922,861,821.12'], dtype='<U32'), 'PPC-Peercoin': array(['24,965,194.32', '25,761,578.25', '24,938,616.54', ..., 'nan',
       'nan', 'nan'], dtype='<U32'), 'BTS-BitShares': array(['19,798,725.48', '19,172,589.49', '19,910,425.06', ...,
      

{'BTC-Bitcoin': array(['1', '1', '1', ..., '1', '1', '1'], dtype='<U32'), 'LTC-Litecoin': array(['2', '2', '2', ..., '8', '9', '8'], dtype='<U32'), 'NXT-Nxt': array(['3', '3', '3', ..., 'nan', 'nan', 'nan'], dtype='<U32'), 'XRP-XRP': array(['4', '4', '4', ..., '3', '3', '3'], dtype='<U32'), 'DASH-Dash': array(['5', '6', '5', ..., '22', '24', '25'], dtype='<U32'), 'PPC-Peercoin': array(['6', '5', '6', ..., 'nan', 'nan', 'nan'], dtype='<U32'), 'BTS-BitShares': array(['7', '7', '7', ..., '132', '134', '130'], dtype='<U32'), 'DOGE-Dogecoin': array(['8', '8', '8', ..., '34', '35', '36'], dtype='<U32'), 'NMC-Namecoin': array(['9', '9', '9', ..., 'nan', 'nan', 'nan'], dtype='<U32'), 'MAID-MaidSafeCoin': array(['10', '10', '10', ..., '186', '184', '186'], dtype='<U32'), 'BLK-BlackCoin': array(['11', '15', '14', ..., 'nan', 'nan', 'nan'], dtype='<U32'), 'XCP-Counterparty': array(['12', '14', '12', ..., 'nan', 'nan', 'nan'], dtype='<U32'), 'XMR-Monero': array(['13', '13', '13', ..., '15', '15', 